# Scrappe Lychee

Website link : https://lycheethelabel.com/

Goal : get all the info product + each review.

+ Gets blogs information about Asian Streetwear's trend

## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

## Fonctions 

In [2]:
def initialize_driver():
    ua = UserAgent()
    user_agent = ua.random

    options = Options()
    options.add_argument('--headless')
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Firefox(options=options)
    return driver

In [3]:
def click_button(driver, xpath):
    try:
        button = driver.find_element(By.XPATH, xpath)
        button.click()
        return True
    except NoSuchElementException:
        print(f"Le bouton avec l'xpath '{xpath}' est introuvable.")
        return False

In [4]:
def scrape_content(driver, xpath, content_type='text', max_retries=3, timeout=15, attribute=None):
    for retry in range(max_retries):
        try:
            # Trouver tous les éléments correspondant à l'XPath fourni
            elements = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, xpath))
            )

            # Extraire le contenu spécifié pour chaque élément
            if content_type == 'text':
                result = [element.text if element.text else None for element in elements]
            elif content_type == 'href':
                result = [element.get_attribute('href') if element.get_attribute('href') else None for element in elements]
            elif content_type == 'src':
                result = [element.get_attribute('src') if element.get_attribute('src') else None for element in elements]
            elif content_type == 'srcset':
                result = [element.get_attribute('srcset') if element.get_attribute('srcset') else None for element in elements]
            elif content_type == 'data-score' or content_type == 'data-content':
                if attribute is not None:
                    result = [element.get_attribute(attribute) if element.get_attribute(attribute) else None for element in elements]
                else:
                    raise ValueError("L'attribut 'attribute' doit être spécifié pour 'data-score' et 'data-content'.")
            else:
                raise ValueError("Type de contenu non pris en charge. Utilisez 'text', 'href', 'src', 'srcset', 'data-score' ou 'data-content'.")

            return result

        except TimeoutException:
            print(f"Timeout de {timeout} secondes atteint. Réessai {retry + 1}/{max_retries}.")

        except NoSuchElementException:
            print(f"Élément non trouvé. Vérifiez votre XPath. Réessai {retry + 1}/{max_retries}.")

        except StaleElementReferenceException:
            print(f"Stale Element Reference Exception. L'élément n'est plus attaché au DOM. Réessai {retry + 1}/{max_retries}.")

        except Exception as e:
            print(f"Une erreur s'est produite : {str(e)}")

    print(f"Impossible de trouver et extraire le contenu après {max_retries} tentatives.")
    return []


In [6]:
def action_aleatoire(driver):
    actions_possibles = ["scroll_full", "scroll_half", "move_to_element"]
    action_choisie = random.choice(actions_possibles)

    # Enregistrer la position actuelle de la page
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    if action_choisie == "scroll_full":
        # Action aléatoire : Faites défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    elif action_choisie == "scroll_half":
        # Action aléatoire : Faites défiler la moitié de la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
    elif action_choisie == "move_to_element":
        # Action aléatoire : Bougez la souris vers un élément aléatoire (par exemple, div de class PriceInformation_classifiedPrice__b-Jae)
        wait = WebDriverWait(driver, 10)
        element_to_move = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'center')))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_move)

    # Revenir à la position enregistrée
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

## Scrappe

In [ ]:
# Initialize the driver
driver = initialize_driver()

# Load the page
driver.get("https://lycheethelabel.com/")

In [ ]:
Plan D'action :

-Blog : cliquer sur div class header-blog := aller dans le a de la div ( le href)

sur chaque page recuperer tout les hrefs de blogs se trouvant dans : div class article__inner container  puis  h2  puis balise a  := recup des hrefs

sur chaque page de blog .   titre = by_css_selector('h1[data-mce-fragment]')
                            partie = by_css_selector('h2[data-mce-fragment]')
                            paragraphe = by_css_selector('p[data-mce-fragment]')
                            img = by_css_selector('div[data-mce-fragment]') recup limage dedans img
                            si presence ul =  by_css_selector('ul[data-mce-fragment]') 
                        sur 14 pages ( possibiltés de threadings)
                    

les urls des pages son : vetements = https://lycheethelabel.com/collections/+ particules de parties  := exemple https://lycheethelabel.com/collections/tops pour les tops
                         https://lycheethelabel.com/pages pour les autres particules =: exemple https://lycheethelabel.com/pages/about-us
je vais lister les différentes possibilités et scrapper dedans. 
je vais regarder chaque nombre de page par page pour savoir combien scrapper ( facilité du  multithread)

Sur page produits :   chaque balise a  class product-link ont un href := recuperer tout les hrefs et naviguer dedans

dans feuille produit :
    nom produit : h1 class title := recup text
    prix : span class current-price theme-money := recup text
    nombre reviews : span class jdgm-prev-badge__text := recup text
    size = recupe le ul de class clickyboxes options--size . parcourir chaque li et recup leur text
    wishlist := span de class swym-fave-count := recup text
    img produit :=   dans div class thumbnails desktop-only on recupe les img class rimage__image fade-in lazyautosizes lazyloaded recuperer le srcset via get attribute
    img_reseau = img class snpt__img  := recuperer src
    detail = div class content tout le texte
    recommendation = div class title = get text
    
    recommendations img = dans chaque div class block-inner-inner on recupe chaque  img class rimage__image fade-in lazyautosizes lazyloaded = img srcet
    reviews := a class jdgm-rev__prod-link  :=  recup href
    y a  acceder := 
    un review = div class product-reviews__review-card  on itere dans chaque pour obtenir
    
    note general = span class metrics-widget__rating := text
    recup:= div review__timestamp := get text
    nom profil = a class vertical-middle := text 
    date_post= div class review__timestamp
    lien_profil =  a class vertical-middle := get attribute href
    image = img class lazy-img review-images__picture-img := srcset 
    note := div class star-rating review__rating := get aria-label
    titre post = strong class review__title  := get text
    commentaire_post = div class review__body  ; on prend le p dedans := get text
    
    à la fin une fois page comme faites := click sur span de class pagination__page-number pagination__page-number--icon material-icons pour aller page comme suivante du meme produit
    
            

In [ ]:
# ebauche scrip pour carousell reseau
# Replace 'next_button_xpath' with the XPath of the button to move to the next image
next_button_xpath = "//button[@id='nextButton']"

# Replace 'carousel_xpath' with the XPath of the element containing the carousel
carousel_xpath = "//div[@class='your-carousel-class']"

# Function to extract image URLs from the current carousel state
def extract_image_urls():
    carousel_element = driver.find_element_by_xpath(carousel_xpath)
    carousel_html = carousel_element.get_attribute('innerHTML')
    return [url.split('"')[1] for url in carousel_html.split('srcset="')[1:]]

# Click the next button and extract image URLs multiple times
num_iterations = 5  # Adjust the number of iterations as needed

for _ in range(num_iterations):
    # Extract and print image URLs
    image_urls = extract_image_urls()
    for index, image_url in enumerate(image_urls):
        print(f"Image {index + 1}: {image_url}")

    # Click the next button to move to the next set of images
    next_button = driver.find_element_by_xpath(next_button_xpath)
    ActionChains(driver).move_to_element(next_button).click().perform()

In [5]:
# Initialize the driver
driver = initialize_driver()

try:
    
    driver.get("https://lycheethelabel.com/")

    # acces blog
    blog_xpath = ".//div[@class='header-blog']/a"
    blog_href = scrape_content(driver, blog_xpath, content_type='href')
    print(blog_href)
    if blog_href:
        # Cliquer sur le premier lien
        driver.get(blog_href[0])
    else:
        print("Aucun lien trouvé.")

   
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ".//div[@class='article__inner container']"))
    )

   
    for page_number in range(1, 15):
        # Construire l'URL de la page suivante
        page_url = f"https://lycheethelabel.com/blogs/blog?page={page_number}"

      
        driver.get(page_url)

        # Récup s liens (href) des blogs
        blog_hrefs = scrape_content(driver, "//div[@class='article__inner container']//h2/a", content_type='href')
        print(blog_hrefs)

        # Parcourir chaque lien 
        for href in blog_hrefs:
            # Charger la page du blog
            driver.get(href)

            
            titre = scrape_content(driver, "//h1[@data-mce-fragment]", content_type='text')
            partie = scrape_content(driver, "//h2[@data-mce-fragment]", content_type='text')
            paragraphe = scrape_content(driver, "//p[@data-mce-fragment]", content_type='text')
            img = scrape_content(driver, "//div[@data-mce-fragment]//img", content_type='src')

            
            ul_content = scrape_content(driver, "//ul[@data-mce-fragment]", content_type='text')

            
            print(f"Titre: {titre}")
            print(f"Partie: {partie}")
            print(f"Paragraphe: {paragraphe}")
            print(f"Image: {img}")
            print(f"Contenu de la liste ul: {ul_content}")
            print("\n" + "="*50 + "\n")

finally:
    # Fermer le navigateur à la fin
    driver.quit()

['https://lycheethelabel.com/blogs/blog']
['https://lycheethelabel.com/blogs/blog/rave-culture-in-korea', 'https://lycheethelabel.com/blogs/blog/how-to-prep-for-a-rave', 'https://lycheethelabel.com/blogs/blog/the-trendiest-japanese-streetwear-fashion-of-the-moment', 'https://lycheethelabel.com/blogs/blog/top-10-unique-asian-streetwear-for-women-online-shopping-sites', 'https://lycheethelabel.com/blogs/blog/most-popular-kpop-fashion-trends', 'https://lycheethelabel.com/blogs/blog/the-top-5-platforms-to-stay-updated-on-the-latest-k-pop-news', 'https://lycheethelabel.com/blogs/blog/bts-unveiled-a-comprehensive-guide-to-the-phenomenal-k-pop-sensation', 'https://lycheethelabel.com/blogs/blog/top-5-kpop-trends', 'https://lycheethelabel.com/blogs/blog/secret-k-pop-spots-seoul-korea', 'https://lycheethelabel.com/blogs/blog/iconic-kpop-boy-groups']
Titre: ['Korean Raves and Everything You Need to Know About It!']
Partie: ['A Historical Glimpse of Korean Rave Scene', 'Influences on Korean Rave C

KeyboardInterrupt: 

### MarketPlace

Prep url :

les liens du sites sont sous le format  : 
    https://lycheethelabel.com/collections/ + suffixe de classe . alors je creer une liste de url que je vais parcourir ( afind deviter interaction , et accelerer le process)
    


In [17]:
# Liste pour stocker les liens
urls = []

# Lien unique
for i in range(1, 6):
    urls.append(f"https://lycheethelabel.com/collections/frontpage?page={i}")

# Lien avec itération de 1 à 5
for i in range(1, 6):
    urls.append(f"https://lycheethelabel.com/collections/tops?page={i}")

# Lien avec itération de 0 à 3 pages
for i in range(4):
    urls.append(f"https://lycheethelabel.com/collections/bottoms?page={i}")

# Lien unique
urls.append("https://lycheethelabel.com/collections/accessories")
for i in range(6):
     urls.append(f"https://lycheethelabel.com/collections/lychee-refreshed?page={i}")
# Lien avec itération de 0 à 5 pages
for i in range(6):
    urls.append(f"https://lycheethelabel.com/collections/bestsellers?page={i}")

# Lien avec itération de 0 à 3 pages
for i in range(4):
    urls.append(f"https://lycheethelabel.com/collections/asian-streetwear?page={i}")

# Lien avec itération de 0 à 5 pages
for i in range(6):
    urls.append(f"https://lycheethelabel.com/collections/k-pop-inspired-fashion?page={i}")

# Lien avec itération de 0 à 3 pages
for i in range(4):
    urls.append(f"https://lycheethelabel.com/collections/meant-to-be?page={i}")

# Lien avec itération de 0 à 4 pages
for i in range(5):
    urls.append(f"https://lycheethelabel.com/collections/rave-ready?page={i}")

# Lien unique
urls.append("https://lycheethelabel.com/collections/skirt-sets")

# Lien avec itération de 0 à 3 pages
for i in range(4):
    urls.append(f"https://lycheethelabel.com/collections/grunge?page={i}")

# Afficher la liste des liens générés
for url in urls:
    print(url)


https://lycheethelabel.com/collections/frontpage?page=1
https://lycheethelabel.com/collections/frontpage?page=2
https://lycheethelabel.com/collections/frontpage?page=3
https://lycheethelabel.com/collections/frontpage?page=4
https://lycheethelabel.com/collections/frontpage?page=5
https://lycheethelabel.com/collections/tops?page=1
https://lycheethelabel.com/collections/tops?page=2
https://lycheethelabel.com/collections/tops?page=3
https://lycheethelabel.com/collections/tops?page=4
https://lycheethelabel.com/collections/tops?page=5
https://lycheethelabel.com/collections/bottoms?page=0
https://lycheethelabel.com/collections/bottoms?page=1
https://lycheethelabel.com/collections/bottoms?page=2
https://lycheethelabel.com/collections/bottoms?page=3
https://lycheethelabel.com/collections/accessories
https://lycheethelabel.com/collections/lychee-refreshed?page=0
https://lycheethelabel.com/collections/lychee-refreshed?page=1
https://lycheethelabel.com/collections/lychee-refreshed?page=2
https://l

In [18]:
# Initialize the driver
driver = initialize_driver()

# Liste pour stocker tous les liens de produits
product_links = []
try:
    for url in urls:
        # Charger la page
        driver.get(url)

        # Attendre que la page se charge
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-link"))
        )

        # Récupérer les liens des produits
        product_hrefs = scrape_content(driver, "//a[@class='product-link']", content_type='href')
        product_links.extend(product_hrefs)

except Exception as e:
    print(f"Une erreur s'est produite : {str(e)}")

finally:
    # Fermer le navigateur à la fin
    driver.quit()

# Afficher la liste des liens des produits
for product_link in product_links:
    print(product_link)

Une erreur s'est produite : Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

https://lycheethelabel.com/products/udon-mesh-sleeve-front-zip-bustier
https://lycheethelabel.com/products/udon-mesh-sleeve-front-zip-bustier
https://lycheethelabel.com/products/pandan-asymmetrical-layered-neck-strap-top
https://lycheethelabel.com/products/pandan-asymmetrical-layered-neck-strap-top
https://lycheethelabel.com/products/mei-fun-plaid-velvet-strap-crop-top
https://lycheethelabel.com/products/mei-fun-plaid-velvet-strap-crop-top
https://lycheethelabel.com/products/napa-buckle-strap-top-and-sleeve-set
https://lycheethelabel.com/products/napa-buckle-strap-top-and-sleeve-set
https://lycheethelabel.com/products/tangyuan-v-cutout-crop-top
https://

In [19]:
len(product_link)

68

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def scrape_product_info(product_url):
    product_info = {}

    try:
        # Load the product page
        driver.get(product_url)

        # Wait for the page to load completely
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'snptwdgt-feed')]"))
        )

        # Retrieve the product name
        product_name_xpath = "//h1[contains(@class, 'title')]"
        product_name = scrape_content(driver, product_name_xpath, content_type='text')[0]
        product_info["Product Name"] = product_name

        # Retrieve the product price
        product_price_xpath = "//span[contains(@class, 'current-price') and contains(@class, 'theme-money')]"
        product_price = scrape_content(driver, product_price_xpath, content_type='text')[0]
        product_info["Product Price"] = product_price

        # Retrieve available sizes
        sizes_xpath = "//ul[contains(@class, 'clickyboxes') and contains(@class, 'options--size')]/li"
        available_sizes = scrape_content(driver, sizes_xpath, content_type='text')
        product_info["Available Sizes"] = available_sizes

        # Retrieve data-srcset of images
        images_xpath = "//img[contains(@class, 'rimage__image fade-in lazyautosizes lazyloaded')]"
        image_srcsets = scrape_content(driver, images_xpath, content_type='srcset')
        product_info["Data-srcset of Images"] = image_srcsets

        # Print the information
        for key, value in product_info.items():
            print(f"{key}: {value}")

    except TimeoutException:
        print("Timeout waiting for the page to load.")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

    # Return the dictionary containing product information
    return product_info




In [31]:
driver.quit()

In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import time

def scrape_comments(url):
    comments_data = []

    try:
        # Load the page
        driver.get(url)

        # Scroll to the first 'jdgm-rev' element using JavaScript
        first_review_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'jdgm-rev'))
        )
        driver.execute_script("arguments[0].scrollIntoView();", first_review_element)

        # Wait for comments to be present on the page
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'jdgm-rev'))
        )

        # Loop to paginate through the pages
        while True:
            # Find all elements matching the 'jdgm-rev' class
            review_elements = driver.find_elements(By.CLASS_NAME, 'jdgm-rev')

            # Iterate through all elements to retrieve information
            for review_element in review_elements:
                # Use the scrape_content function to retrieve information specific to each element
                product_link = scrape_content(review_element, ".//a[contains(@class, 'jdgm-rev__prod-link')]")
                author = scrape_content(review_element, ".//span[contains(@class, 'jdgm-rev__author')]")
                title_com = scrape_content(review_element, ".//b[contains(@class, 'jdgm-rev__title')]")
                comment = scrape_content(review_element, ".//div[contains(@class, 'jdgm-rev__body')]")
                rating_com = scrape_content(review_element, ".//span[contains(@class, 'jdgm-rev__rating')]", content_type='data-score', attribute='data-score')
                timestamp = scrape_content(review_element, ".//span[contains(@class, 'jdgm-rev__timestamp')]", content_type='data-content', attribute='data-content')
                img_coms = scrape_content(review_element, ".//img[contains(@class, 'jdgm-rev__pic-img')]", content_type='src')

                # Append the information to the list
                comments_data.append({
                    "Product": product_link,
                    "Author": author,
                    "Rating": rating_com,
                    "Title of the Comment": title_com,
                    "Comment": comment,
                    "Timestamp": timestamp,
                    "Comment Image": img_coms
                })

            # Try to click on the "Next Page" button
            try:
                next_page_button = driver.find_element(By.CLASS_NAME, 'jdgm-paginate__next-page')

                # Scroll to the element using JavaScript
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)

                # Click the "Next Page" button
                next_page_button.click()

                # Wait for the "Next Page" button to become invisible
                WebDriverWait(driver, 15).until_not(
                    EC.visibility_of(next_page_button)
                )

                # Pause for 5 seconds
                time.sleep(5)

            except NoSuchElementException:
                # The button is no longer present, exit the loop
                print("There is no more next page.")
                break
            except ElementNotInteractableException:
                # The element is not currently interactive, exit the loop
                print("The element is not currently interactive. End of pagination.")
                break

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    print(comments_data)
    # Return the list of comment data
    return comments_data


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_carousel_images(driver, carousel_xpath):
    # XPath for the div containing the carousel
    carousel_div_xpath = ".//div[@class='snptwdgt-feed__items-inner snptwdgt-container-liner snptslide-initialized snptslide-slider']"

    # XPath for the next button
    next_button_xpath = f"{carousel_div_xpath}/button"

    # Wait for the carousel to be present
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, carousel_div_xpath))
    )

    try:
        # Retrieve all images in the carousel
        all_images = driver.find_elements(By.XPATH, carousel_div_xpath + "//img")

        # Collect the src of each image
        image_srcs = [image.get_attribute('src') for image in all_images]

        # Print the src of the images
        print("Src des images dans le carrousel:")
        for src in image_srcs:
            print(src)

        # Click the next button to move to the next image
        for image in all_images[:-1]:
            # Scroll down to bring the element into view
            ActionChains(driver).key_down(Keys.ARROW_DOWN).key_up(Keys.ARROW_DOWN).perform()
            time.sleep(3)

            # Hover over the next button to make it visible
            next_button = driver.find_element(By.XPATH, next_button_xpath)
            ActionChains(driver).move_to_element(next_button).perform()

            # Click the next button
            ActionChains(driver).click(next_button).perform()

            # Wait for the carousel to update
            WebDriverWait(driver, 30).until(EC.staleness_of(image))

            # Collect the src of the next image
            image_src = image.get_attribute('src')
            print(f"Src de l'image suivante : {image_src}")

    except Exception as e:
        print(f"Une erreur s'est produite : {str(e)}")


# Code sale

In [46]:
### Generation urls du sites 
# Liste pour stocker les liens
urls = []
for i in range(1, 6):
    urls.append(f"https://lycheethelabel.com/collections/frontpage?page={i}")
for i in range(1, 6):
    urls.append(f"https://lycheethelabel.com/collections/tops?page={i}")
for i in range(6):
    urls.append(f"https://lycheethelabel.com/collections/lychee-refreshed?page={i}")
    urls.append(f"https://lycheethelabel.com/collections/bestsellers?page={i}")
    urls.append(f"https://lycheethelabel.com/collections/k-pop-inspired-fashion?page={i}")

# Lien avec itération de 0 à 3 pages
for i in range(4):
    urls.append(f"https://lycheethelabel.com/collections/meant-to-be?page={i}")
    urls.append(f"https://lycheethelabel.com/collections/grunge?page={i}")
    urls.append(f"https://lycheethelabel.com/collections/asian-streetwear?page={i}")
    urls.append(f"https://lycheethelabel.com/collections/bottoms?page={i}")
for i in range(5):
    urls.append(f"https://lycheethelabel.com/collections/rave-ready?page={i}")
urls.append("https://lycheethelabel.com/collections/skirt-sets")
urls.append("https://lycheethelabel.com/collections/accessories")


################## Recup fiche produit
# Initialize the driver
driver = initialize_driver()

# Liste pour stocker tous les liens de produits
product_links = []
try:
    for url in urls:
        # Charger la page
        driver.get(url)

        # Attendre que la page se charge
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-link"))
        )

        # Récupérer les liens des produits
        product_hrefs = scrape_content(driver, "//a[@class='product-link']", content_type='href')
        product_links.extend(product_hrefs)

except Exception as e:
    print(f"Une erreur s'est produite : {str(e)}")



for product_url in product_links:
    # Charger la page du produit
    driver.get(product_url)

    try:
        # Attendre que la page se charge complètement
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'snptwdgt-feed')]"))
        )

        # Récupérer les informations du produit
        product_name_xpath = "//h1[contains(@class, 'title')]"
        product_name = scrape_content(driver, product_name_xpath, content_type='text')[0]
        print(f"Product Name: {product_name}")

        product_price_xpath = "//span[contains(@class, 'current-price') and contains(@class, 'theme-money')]"
        product_price = scrape_content(driver, product_price_xpath, content_type='text')[0]
        print(f"Product Price: {product_price}")

        sizes_xpath = "//ul[contains(@class, 'clickyboxes') and contains(@class, 'options--size')]/li"
        available_sizes = scrape_content(driver, sizes_xpath, content_type='text')
        print(f"Available Sizes: {available_sizes}")

        images_xpath = "//img[contains(@class, 'rimage__image fade-in lazyautosizes lazyloaded')]"
        image_srcsets = scrape_content(driver, images_xpath, content_type='srcset')
        print(f"Data-srcset of Images: {image_srcsets}")

        # Attendre que les commentaires soient présents sur la page
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'jdgm-rev'))
        )

        # Boucle pour paginer à travers les pages de commentaires
        while True:
            review_elements = driver.find_elements(By.CLASS_NAME, 'jdgm-rev')

            for review_element in review_elements:
                product_link = scrape_content(review_element, ".//a[contains(@class, 'jdgm-rev__prod-link')]")
                author = scrape_content(review_element, ".//span[contains(@class, 'jdgm-rev__author')]")
                title_com = scrape_content(review_element, ".//b[contains(@class, 'jdgm-rev__title')]")
                comment = scrape_content(review_element, ".//div[contains(@class, 'jdgm-rev__body')]")
                rating_com = scrape_content(review_element, ".//span[contains(@class, 'jdgm-rev__rating')]", content_type='data-score', attribute='data-score')
                timestamp = scrape_content(review_element, ".//span[contains(@class, 'jdgm-rev__timestamp')]", content_type='data-content', attribute='data-content')
                img_coms = scrape_content(review_element, ".//img[contains(@class, 'jdgm-rev__pic-img')]", content_type='src')

                # Imprimer les informations
                print(f"Produit: {product_link}")
                print(f"Auteur: {author}")
                print(f"Score: {rating_com}")
                print(f"Titre du commentaire: {title_com}")
                print(f"Commentaire: {comment}")
                print(f"Timestamp: {timestamp}")
                print(f"Image du commentaire: {img_coms}")
                print("-----------")

            try:
                next_page_button = driver.find_element(By.CLASS_NAME, 'jdgm-paginate__next-page')
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                next_page_button.click()
                WebDriverWait(driver, 15).until_not(EC.visibility_of(next_page_button))
                time.sleep(5)

            except NoSuchElementException:
                print("Il n'y a plus de page suivante.")
                break
            except ElementNotInteractableException:
                print("L'élément n'est pas actuellement interactif. Fin de la pagination.")
                break

    except Exception as e:
        print(f"Une erreur s'est produite : {str(e)}")

    finally:
        # Fermer le navigateur à la fin de chaque itération
        driver.quit()

Une erreur s'est produite : Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Product Name: Udon Mesh Sleeve Front-Zip Bustier
Product Price: $39.90
Available Sizes: ['SMALL', 'MEDIUM', 'LARGE', 'X-LARGE']
Data-srcset of Images: ['//lycheethelabel.com/cdn/shop/files/udon-mesh-bustier-crop_180x.jpg?v=1687965242 180w, //lycheethelabel.com/cdn/shop/files/udon-mesh-bustier-crop_220x.jpg?v=1687965242 220w, //lycheethelabel.com/cdn/shop/files/udon-mesh-bustier-crop_300x.jpg?v=1687965242 300w, //lycheethelabel.com/cdn/shop/files/udon-mesh-bustier-crop_360x.jpg?v=1687965242 360w, //lycheethelabel.com/cdn/shop/files/udon-mesh-bustier-crop_460x.jpg?v=1687965242 460w, //lycheethelabel.com/cdn/shop/files/udon-mesh-bustier-crop_540x.jpg?v=168

MaxRetryError: HTTPConnectionPool(host='localhost', port=60624): Max retries exceeded with url: /session/7d6374db-a7f5-4951-9014-a42641ba85c6/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E86942A970>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

# OUI CODE

In [ ]:
urls = []

# generation url
for category in ['frontpage', 'tops']:
    for i in range(1, 6):
        urls.append(f"https://lycheethelabel.com/collections/{category}?page={i}")


for category in ['lychee-refreshed', 'bestsellers', 'k-pop-inspired-fashion']:
    for i in range(6):
        urls.append(f"https://lycheethelabel.com/collections/{category}?page={i}")


for category in ['meant-to-be', 'grunge', 'asian-streetwear', 'bottoms']:
    for i in range(4):
        urls.append(f"https://lycheethelabel.com/collections/{category}?page={i}")


for i in range(5):
    urls.append(f"https://lycheethelabel.com/collections/rave-ready?page={i}")


urls.extend([
    "https://lycheethelabel.com/collections/skirt-sets",
    "https://lycheethelabel.com/collections/accessories"
])

# Ini driver
driver = initialize_driver()

# List  product links
product_links = []
try:
    for url in urls:
        
        driver.get(url)
        print(url)
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-link"))
        )

        # recup  links
        product_hrefs = scrape_content(driver, "//a[@class='product-link']", content_type='href')
        product_links.extend(product_hrefs)

except Exception as e:
    print(f"An error occurred: {str(e)}")

# itere dans chaque produit avec fontion
for product_url in product_links:
    print(product_url)
    url_product= product_url
    print('2')
    product_info = scrape_product_info(product_url)
    print(scrape_product_info)
    
    img_carou = scrape_carousel_images(driver, "//div[@class='snptwdgt-feed snptwdgt-container snpt_widget--product_page_gallery snpt_widget--carousel snpt_widget--itm-hov-zoom snptwdgt_viewpreloaded']")
    print(scrape_carousel_images)
    print('4')
    comments = scrape_comments(product_url)
    print(scrape_comments)
    print('5')
# Close the browser at the end
driver.quit()

https://lycheethelabel.com/collections/frontpage?page=1
https://lycheethelabel.com/collections/frontpage?page=2
https://lycheethelabel.com/collections/frontpage?page=3
https://lycheethelabel.com/collections/frontpage?page=4
https://lycheethelabel.com/collections/frontpage?page=5
https://lycheethelabel.com/collections/tops?page=1
https://lycheethelabel.com/collections/tops?page=2
https://lycheethelabel.com/collections/tops?page=3
https://lycheethelabel.com/collections/tops?page=4
An error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

https://lycheethelabel.com/products/udon-mesh-sleeve-front-zip-bustier
2
Product Name: Udon Mesh Sleeve Front-Zip Bustier
Product Price: $39.90
Available Sizes: ['SMALL', 'MEDIUM', 'LARG